In [ ]:
import boto3
import os
from dotenv import load_dotenv
from mypy_boto3_s3 import S3Client
from botocore.config import Config
import datetime
import uuid
import base64

load_dotenv(dotenv_path=os.path.join("../.env"), override=True)

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = str(os.environ.get("AWS_BUCKET_NAME"))
REGION_NAME = str(os.environ.get("AWS_REGION_NAME"))

req = {
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY_ID,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_ACCESS_KEY,
    "BUCKET_NAME": BUCKET_NAME,
    "REGION_NAME": REGION_NAME,
}

for key, value in req.items():
    if value is None:
        raise ValueError(f"{key} is not set in the environment variables.")


In [ ]:
TEST_IMAGE_PATH = 'img/test-img-sq-person-1.jpg'

s3_client: S3Client = boto3.client(
    's3',
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    endpoint_url=f"https://s3.{REGION_NAME}.amazonaws.com", # 2025-11-12 I don't know why but for ap.northeast-2, this is needed.
    config=Config(signature_version="s3v4", s3={"addressing_style": "virtual"}),
) # type: ignore

# GEnerate object name based on timstamp + random string
import datetime
import uuid
# Generate a unique filename
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d%H%M") # Current timestamp
random_string = uuid.uuid4().hex[:12]  # 8-character random string
filename, file_extension = os.path.splitext(os.path.basename(TEST_IMAGE_PATH)) # Extract the filename and extension
OBJECT_NAME = f"{timestamp}_{random_string}{file_extension}"


try:
    with open(TEST_IMAGE_PATH, 'rb') as file:
        s3_client.upload_fileobj(file, BUCKET_NAME, OBJECT_NAME)

    # s3_client.upload_file(UPLOAD_FILE, BUCKET_NAME, OBJECT_NAME) # type: ignore
    print(f"File '{TEST_IMAGE_PATH}' uploaded to '{BUCKET_NAME}/{OBJECT_NAME}'")

    url = s3_client.generate_presigned_url(
        'get_object',
        Params={
            'Bucket': BUCKET_NAME,
            'Key': OBJECT_NAME,
            'ResponseContentType': 'image/jpeg'
        },
        ExpiresIn=3600
    )
    print(f"Presigned URL: {url}")

except Exception as e:
    print(f"Error: {e}")

In [ ]:
import boto3
from botocore.config import Config
import os
from dotenv import load_dotenv

# Load variables from .env file into environment
load_dotenv()

# --- Configuration ---
# Replace with a real object key from your bucket
OBJECT_KEY = "202401011200_someimage.jpg"
BUCKET_NAME = 'bucket-mealmap-1'
REGION_NAME = 'ap-northeast-2'

# --- Sanity Check: Verify what credentials the script is seeing ---
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

print(f"Script sees Access Key ID: {access_key}")
if not access_key or not secret_key:
    print("FATAL: AWS credentials not found in .env file or environment!")
    exit()

# --- Boto3 Client Setup ---
s3_config = Config(
    region_name=REGION_NAME,
    signature_version='s3v4',
)

s3_client = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=s3_config
)

# --- Generate the URL ---
try:
    print(f"\nAttempting to generate presigned URL for s3://{BUCKET_NAME}/{OBJECT_KEY}")
    url = s3_client.generate_presigned_url(
        'get_object',
        Params={
            'Bucket': BUCKET_NAME,
            'Key': OBJECT_KEY,
            'ResponseContentType': 'image/jpeg'
        },
        ExpiresIn=3600
    )
    print("\n--- SUCCESS! ---")
    print(f"Presigned URL: {url}")

except Exception as e:
    print(f"\n--- ERROR ---")
    print(f"An error occurred: {e}")


In [ ]:
import boto3
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path=os.path.join("../.env"), override=True)

# --- 1. Load configuration from environment ---
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
BUCKET_NAME = os.environ.get('AWS_BUCKET_NAME')
REGION_NAME = os.environ.get('AWS_REGION_NAME')

# A known object key that already exists in your bucket for testing
OBJECT_KEY = "202401011200_someimage.jpg" 

# --- 2. Correctly initialize the S3 client ---
# The region_name is passed as a direct top-level argument.
# This ensures both the signature AND the URL hostname are built for the correct region.
s3_client = boto3.client(
    's3',
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# --- 3. Generate the presigned URL ---
try:
    print(f"Generating URL for s3://{BUCKET_NAME}/{OBJECT_KEY} in region {REGION_NAME}")
    
    url = s3_client.generate_presigned_url(
        'get_object',
        Params={
            'Bucket': BUCKET_NAME,
            'Key': OBJECT_KEY,
            'ResponseContentType': 'image/jpeg'
        },
        ExpiresIn=3600  # URL expires in 1 hour
    )
    
    print("\n--- SUCCESS! ---")
    print("The generated URL will now have the correct hostname and will work.")
    print(f"\nURL: {url}")

except Exception as e:
    print(f"\nAn error occurred: {e}")



In [ ]:
os.environ.get('REGION_NAME')

In [ ]:
# filename: presign.py
import os
import boto3
from dotenv import load_dotenv
from botocore.config import Config

load_dotenv(dotenv_path=os.path.join("../.env"), override=True)

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")
REGION_NAME = os.getenv("AWS_REGION_NAME")
OBJECT_KEY = "202511120057_d641ec5b3d24.jpg"

session = boto3.session.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

# Force regional endpoint + v4 signing + virtual-hosted style
s3 = session.client(
    "s3",
    endpoint_url=f"https://s3.{REGION_NAME}.amazonaws.com",
    config=Config(signature_version="s3v4", s3={"addressing_style": "virtual"}),
)

print("Endpoint URL used by client:", s3.meta.endpoint_url)  # should be https://s3.ap-northeast-2.amazonaws.com

url = s3.generate_presigned_url(
    "get_object",
    Params={"Bucket": BUCKET_NAME, "Key": OBJECT_KEY, "ResponseContentType": "image/jpeg"},
    ExpiresIn=3600,
)

print("Presigned URL:", url)
